In [ ]:
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

class Retangulo:
    def __init__(self, pontos, eh_vazamento=False):
        self.pontos = pontos
        self.eh_vazamento = eh_vazamento

        self.base = abs(pontos[1][0] - pontos[0][0])  # Diferença entre x1 e x2
        self.altura = abs(pontos[2][1] - pontos[1][1])  # Diferença entre y2 e y1
        self.area = self.base * self.altura

        # Calcula o centroide do retângulo
        self.centroide = self.calcula_centroide_retangulo()

        self.Ix_ = 1 / 12 * self.base * self.altura**3
        self.Iy_ = 1 / 12 * self.altura * self.base**3

    def calcula_centroide_retangulo(self):
        """
        Calcula o centroide de um único retângulo.
        """
        x_medio = (self.pontos[0][0] + self.pontos[1][0]) / 2  # Média dos x
        y_medio = (self.pontos[0][1] + self.pontos[2][1]) / 2  # Média dos y
        return (x_medio, y_medio)

def captura_retangulos(eh_vazamento=False):
    num_retangulos = int(input("Insira o número de retângulos existentes na sua figura: "))
    retangulos = []

    for i in range(num_retangulos):
        pontos = []
        print()
        j = 0
        while j < 4:
            entrada = input(f"Digite um ponto no formato x,y correspondente ao {j+1}º vértice do seu {i+1}º retângulo (m): ")

            try:
                x, y = map(float, entrada.split(','))
                pontos.append((x, y))
                j += 1
            except ValueError:
                print("Entrada inválida! Digite no formato x,y")

        retangulo = Retangulo(ordenar_vertices(pontos), eh_vazamento)
        retangulos.append(retangulo)

    return retangulos

def ordenar_vertices(vertices):
    # Ordena os vértices primeiro por y (ascendente) e depois por x (ascendente)
    vertices_ordenados = sorted(vertices, key=lambda v: (v[1], v[0]))

    # Separa os vértices inferiores e superiores
    inferiores = sorted(vertices_ordenados[:2], key=lambda v: v[0])
    superiores = sorted(vertices_ordenados[2:], key=lambda v: -v[0])

    # Junta os vértices na ordem correta
    return [inferiores[0], inferiores[1], superiores[0], superiores[1]]

def calcula_centroide(retangulos):
    """
    Calcula o centroide de uma figura composta por retângulos.
    O centroide é a média ponderada dos centroides de cada retângulo,
    onde o peso é a área de cada retângulo.
    """
    somatorio_area = 0
    somatorio_x = 0
    somatorio_y = 0

    for retangulo in retangulos:
        if retangulo.eh_vazamento:
            area = -retangulo.area  # Área negativa para vazamentos
        else:
            area = retangulo.area

        # Centroide do retângulo
        x_medio, y_medio = retangulo.centroide

        # Acumular para o cálculo do centroide total
        somatorio_area += area
        somatorio_x += x_medio * area
        somatorio_y += y_medio * area

    # Calcular o centroide total
    if somatorio_area == 0:
        raise ValueError("A área total da figura é zero. Não é possível calcular o centroide.")

    x_centroide = somatorio_x / somatorio_area
    y_centroide = somatorio_y / somatorio_area

    return (x_centroide, y_centroide)

def calcula_momento_inercia(retangulos, x_centroide, y_centroide):
    """
    Calcula o momento de inércia de uma figura composta por múltiplos retângulos.
    Fórmulas:
        Ix = Σ (Ix_ + A * dy²)
        Iy = Σ (Iy_ + A * dx²)
    onde:
        - Ix_ e Iy_ são os momentos de inércia dos retângulos individuais em seus próprios centroides
        - A é a área de cada retângulo (positiva para retângulos principais, negativa para vazamentos)
        - dx e dy são as distâncias entre os centroides individuais e o centroide da figura total
    """
    Ix_total = 0
    Iy_total = 0
    dx = 0
    dy = 0

    for retangulo in retangulos:
        # Área do retângulo (positiva para retângulos principais, negativa para vazamentos)
        area = retangulo.area if not retangulo.eh_vazamento else -retangulo.area

        # Centroide do retângulo
        x_medio, y_medio = retangulo.centroide

        # Distâncias ao centroide da figura
        dx = x_medio - x_centroide
        dy = y_medio - y_centroide

        # Cálculo do momento de inércia usando o teorema dos eixos paralelos
        if not retangulo.eh_vazamento:
            Ix_total += retangulo.Ix_ + abs(area) * dy**2  # Usamos abs(area) para garantir que o sinal não afete o cálculo
            Iy_total += retangulo.Iy_ + abs(area) * dx**2

        if retangulo.eh_vazamento:
            Ix_total -= retangulo.Ix_ + abs(area) * dy**2  # Usamos abs(area) para garantir que o sinal não afete o cálculo
            Iy_total -= retangulo.Iy_ + abs(area) * dx**2

    return Ix_total, Iy_total

def plotagem(retangulos, centroide=None, Ix=None, Iy=None):
    plt.figure()
    for retangulo in retangulos:
        # Define a cor com base se é um vazamento ou não
        if retangulo.eh_vazamento:
            cor = 'white'  # Vazamentos são plotados como brancos
            borda = 'grey'  # Borda preta para destacar o vazamento
        else:
            cor = 'purple'  # Retângulos principais são plotados em azul
            borda = 'purple'

        # Desenha os retângulos conectando os pontos
        pontos = retangulo.pontos
        pontos.append(pontos[0])  # Fechar o retângulo conectando o último ponto ao primeiro
        x_coords, y_coords = zip(*pontos)
        plt.plot(x_coords, y_coords, marker='o', color=borda, label='Vazamento' if retangulo.eh_vazamento else 'Retângulo Principal')
        plt.fill(x_coords, y_coords, color=cor, edgecolor=borda, alpha=0.8 if not retangulo.eh_vazamento else 1.0)  # Preenchimento

    # Plotar o centroide, se fornecido
    if centroide:
        plt.scatter(centroide[0], centroide[1], color='green', marker='x', label='Centroide', s=100, linewidths=2)

        # Exibir os valores do centroide e momentos de inércia no gráfico
        texto_centroide = f'Centroide: ({centroide[0]:.2f}, {centroide[1]:.2f})'
        texto_Ix = f'Ix: {Ix:.2e}'
        texto_Iy = f'Iy: {Iy:.2e}'

        plt.text(0.02, 0.98, texto_centroide, transform=plt.gca().transAxes, fontsize=10, verticalalignment='top', bbox=dict(facecolor='white', alpha=0.8))
        plt.text(0.02, 0.92, texto_Ix, transform=plt.gca().transAxes, fontsize=10, verticalalignment='top', bbox=dict(facecolor='white', alpha=0.8))
        plt.text(0.02, 0.86, texto_Iy, transform=plt.gca().transAxes, fontsize=10, verticalalignment='top', bbox=dict(facecolor='white', alpha=0.8))

    plt.xlabel('X')
    plt.ylabel('Y')
    plt.title('Configuração Final da Figura')
    plt.grid(True)
    plt.legend()
    plt.show()

def menu_retangulos(retangulos):
    print("\n1. Adicionar figura principal")
    print("2. Adicionar Vazamento")
    print("3. Finalizar e Calcular")

    opcao = int(input("\nInsira a opção desejada: "))

    if opcao == 1:
        retangulos.extend(captura_retangulos())  # Adiciona retângulos à lista principal
        #plotagem(retangulos)
        return menu_retangulos(retangulos)

    elif opcao == 2:
        retangulos.extend(captura_retangulos(eh_vazamento=True))  # Adiciona vazamentos
        #plotagem(retangulos)
        return menu_retangulos(retangulos)

    elif opcao == 3:
        if not retangulos:
          print("Nenhum retângulo foi adicionado. Adicione retângulos antes de calcular o centroide.")
        else:
          centroide = calcula_centroide(retangulos)

          Ix, Iy = calcula_momento_inercia(retangulos, centroide[0], centroide[1])
          print(f"\nCentroide: {centroide[0]:.2f} m, {centroide[1]:.2f} m")
          print(f"\nMomento de Inércia em X: {Ix:.2e} m⁴")
          print(f"Momento de Inércia em Y: {Iy:.2e} m⁴")
          print()

          plotagem(retangulos, centroide, Ix, Iy)
          return

    else:
        print("Opção inválida. Tente novamente.")
        return menu_retangulos(retangulos)

retangulos = []  # Lista de retângulos principal
menu_retangulos(retangulos)